In [270]:
import pandas as pd
import inflection
import numpy as np
import plotly.express as px
import mercury as mr
import warnings

In [271]:
warnings.filterwarnings('ignore')
show_code = mr.Checkbox(value=False, label="Show/Hide code")
app = mr.App(title="Nplace notebook", 
            description="You can show and hide code in this app",
            show_code=show_code.value)

mercury.Checkbox

mercury.App

# 0 - FUNCTIONS

In [272]:
def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df

def country_name(country_id):
    COUNTRIES = {
    1: "India",
    14: "Australia",
    30: "Brazil",
    37: "Canada",
    94: "Indonesia",
    148: "New Zeland",
    162: "Philippines",
    166: "Qatar",
    184: "Singapure",
    189: "South Africa",
    191: "Sri Lanka",
    208: "Turkey",
    214: "United Arab Emirates",
    215: "England",
    216: "United States of America",
    }
    return COUNTRIES[country_id]

def create_price_type(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"
    
def color_name(color_code):
    COLORS = {
    "3F7E00": "darkgreen",
    "5BA829": "green",
    "9ACD32": "lightgreen",
    "CDD614": "orange",
    "FFBA00": "red",
    "CBCBC8": "darkred",
    "FF7800": "darkred",
    }
    return COLORS[color_code]

# 1 - DATA CLEANING

In [273]:
df_raw = pd.read_csv('raw_data/zomato.csv')

In [274]:
df1 = df_raw.copy()

In [275]:
#-----------------------------------------
# 1.1 - Rename Columns
#-----------------------------------------
df1 = rename_columns(df1)


In [276]:
#-----------------------------------------
# 1.2 - Clean Na
#-----------------------------------------
df1['cuisines'] = df1['cuisines'].fillna('Unspecified')

In [277]:
#-----------------------------------------
# 1.3 - Drop Duplicates
#-----------------------------------------
df1 = df1.drop_duplicates()

In [278]:
#-----------------------------------------
# 1.4 - Business Restrictions
#-----------------------------------------

#* 1 - Only one cuisine type is considered per restaurant
df1["cuisines"] = df1.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])

#* 2 - Select the restaurant with lowest id if more than 1 answer is found

# 2 - BUSINESS QUESTIONS ANSWERS

# 2.1 - General View

### How many restaurants are registered?

In [279]:
df_aux = len(df1)
df_aux

6942

### How many unique countries are registered?

In [280]:
df1['country_code'] = df1['country_code'].apply(lambda x: country_name(x))
df1 = df1.rename(columns = {'country_code':'country_name'})
df_aux = df1['country_name'].nunique()
df_aux

15

### How many unique cities are registered?

In [281]:
df_aux = df1['city'].nunique()
df_aux

125

### How many total votes are registered?

In [282]:
df_aux = df1['votes'].sum()
df_aux = '{0:,}'.format(df_aux)
print(df_aux)

4,195,634


### How many cuisine types are registered?

In [283]:
df_aux = df1['cuisines'].nunique()
df_aux

166

# 2.2 - Countries view

### Which country has the most registered cities?

In [284]:
df_aux = df1[['country_name', 'city']].groupby('country_name').nunique().sort_values('city', ascending=False).reset_index()
fig = px.bar(df_aux, x='country_name', y='city', text_auto=True, title='Registered cities', labels={'country_name': 'Countries', 'city':'Cities'}) 
fig.show()

### Which country has the most registered restaurants?

In [285]:
df_aux = df1[['country_name', 'restaurant_id']].groupby('country_name').nunique().sort_values('restaurant_id', ascending=False).reset_index()
fig = px.bar(df_aux, x='country_name', y='restaurant_id', text_auto=True, title='Registered restaurants', labels={'country_name': 'Countries', 'restaurant_id':'Restaurants'}) 
fig.show()

### Which country has the most registered restaurants with price range 4

In [286]:
df_aux = df1[['country_name', 'restaurant_id', 'price_range']].groupby(['country_name', 'price_range']).nunique().sort_values('restaurant_id', ascending=False).reset_index()
df_aux = df_aux[df_aux['price_range'] == 4]
df_aux = df_aux[['country_name', 'restaurant_id']]
fig = px.bar(df_aux, x='country_name', y='restaurant_id',text_auto=True, title='Registered restaurants with price range 4', labels={'country_name': 'Countries', 'restaurant_id':'Restaurants'}) 
fig.show()

### Which country has the most distinct cuisines types?

In [287]:
df_aux = df1[['country_name', 'cuisines']].groupby('country_name').nunique().sort_values('cuisines', ascending=False).reset_index()
fig = px.bar(df_aux, x='country_name', y='cuisines',text_auto=True, title='Restaurants distinct cuisines', labels={'country_name': 'Countries', 'cuisines':'Cuisines'}) 
fig.show()

### Which country has the highest rating count?

In [288]:
df_aux = df1[['country_name', 'votes']].groupby('country_name').sum().sort_values('votes', ascending=False).reset_index()
fig = px.bar(df_aux, x='country_name', y='votes',text_auto=True, title='Number of restaurants votes received', labels={'country_name': 'Countries', 'votes':'Votes'}) 
fig.show()

### Which country has the most restaurants that do deliveries?

In [289]:
df_aux = df1[['country_name', 'restaurant_id', 'is_delivering_now']].groupby(['country_name', 'is_delivering_now']).nunique().sort_values('restaurant_id', ascending=False).reset_index()
df_aux = df_aux[df_aux['is_delivering_now'] == 1]
df_aux = df_aux[['country_name', 'restaurant_id']]
fig = px.bar(df_aux, x='country_name', y='restaurant_id',text_auto=True, title='Number of restaurants that delivery', labels={'country_name': 'Countries', 'restaurant_id':'Restaurants'}) 
fig.show()

### Which country has the most restaurants that has table booking?

In [290]:
df_aux = df1[['country_name', 'restaurant_id', 'has_table_booking']].groupby(['country_name', 'has_table_booking']).nunique().sort_values('restaurant_id', ascending=False).reset_index()
df_aux = df_aux[df_aux['has_table_booking'] == 1]
df_aux = df_aux[['country_name', 'restaurant_id']]
fig = px.bar(df_aux, x='country_name', y='restaurant_id',text_auto=True, title='Number of restaurants that has table booking', labels={'country_name': 'Countries', 'restaurant_id':'Restaurants'}) 
fig.show()

### Which country has the highest mean votes count registered?

In [291]:
df_aux = df1[['country_name', 'votes']].groupby('country_name').mean().sort_values('votes', ascending=False).reset_index()
fig = px.bar(df_aux, x='country_name', y='votes',text_auto=True, title='Mean votes count per country', labels={'country_name': 'Countries', 'votes':'Mean votes count'}) 
fig.show()

### Which country has the best and worst mean rating registered?

In [292]:
df_aux = df1[df1['rating_text'] != 'Not rated']
df_aux = df_aux[['country_name', 'aggregate_rating']].groupby('country_name').mean().sort_values('aggregate_rating', ascending=False).reset_index()
fig = px.bar(df_aux, x='country_name', y='aggregate_rating',text_auto=True, title='Mean rating per country', labels={'country_name': 'Countries', 'aggregate_rating':'Mean Rating'}) 
fig.show()

### What is the average cost for two per country?

In [293]:
df_aux = df1[['country_name', 'average_cost_for_two']].groupby('country_name').mean().sort_values('average_cost_for_two', ascending=False).reset_index()
fig = px.bar(df_aux, x='country_name', y='average_cost_for_two',text_auto=True, title='Average cost for two per country', labels={'country_name': 'Countries', 'average_cost_for_two':'Average cost for two'}) 
fig.show()

# 2.3 - Cities View

### Which city has the largest number of restaurants registered?

In [294]:
df_aux = df1[['city', 'restaurant_id']].groupby('city').nunique().sort_values('restaurant_id', ascending=False).reset_index()
fig = px.bar(df_aux, x='city', y='restaurant_id',text_auto=True, title='Restaurants registered per city', labels={'city': 'Cities', 'restaurant_id':'Restaurants'}) 
fig.show()


### Wich city has the largest number of restaurants with mean rating above 4?

In [295]:
df_aux = df1[df1['votes'] != 0]
df_aux = df_aux[df1['aggregate_rating'] > 4][['city', 'restaurant_id']]
df_aux = df_aux[['city', 'restaurant_id']].groupby('city').nunique().sort_values('restaurant_id', ascending=False).reset_index()
fig = px.bar(df_aux, x='city', y='restaurant_id',text_auto=True, title='Restaurants with mean rating above 4 per city', labels={'city': 'Cities', 'restaurant_id':'Restaurants'}) 
fig.show()

### Wich city has the largest number of restaurants with mean rating below 2.5?

In [296]:
df_aux = df1[df1['votes'] != 0]
df_aux = df_aux[df1['aggregate_rating'] < 2.5][['city', 'restaurant_id']]
df_aux = df_aux[['city', 'restaurant_id']].groupby('city').nunique().sort_values('restaurant_id', ascending=False).reset_index()
fig = px.bar(df_aux, x='city', y='restaurant_id',text_auto=True, title='Restaurants with mean rating below 2.5 per city', labels={'city': 'Cities', 'restaurant_id':'Restaurants'}) 
fig.show()

### Wich city has the most expansive average cost for two?

In [297]:
df_aux = df1[['city', 'average_cost_for_two']].groupby('city').mean().sort_values('average_cost_for_two', ascending=False).reset_index().head(30)
fig = px.bar(df_aux, x='city', y='average_cost_for_two',text_auto=True, title='Average cost for two per city', labels={'city': 'Cities', 'average_cost_for_two':'Average cost for two'}) 
fig.show()

### Wich city has the largest number of distinct cuisines?

In [298]:
df_aux = df1[['city', 'cuisines']].groupby('city').nunique().sort_values('cuisines', ascending=False).reset_index()
fig = px.bar(df_aux, x='city', y='cuisines',text_auto=True, title='Distinct cuisines per city', labels={'city': 'Cities', 'cuisines':'Cuisines'}) 
fig.show()

In [299]:
# Wich city has the largest number of restaurants that accepts table booking?

df_aux = df1[['city', 'restaurant_id', 'has_table_booking']].groupby(['city', 'has_table_booking']).nunique().sort_values('restaurant_id', ascending=False).reset_index()
df_aux = df_aux[df_aux['has_table_booking'] == 1]
fig = px.bar(df_aux, x='city', y='restaurant_id',text_auto=True, title='Restaurants that accepts table booking per city', labels={'city': 'Cities', 'restaurant_id':'Restaurants'}) 
fig.show()

### Wich city has the largest number of restaurants that do deliveries?

In [300]:
df_aux = df1[['city', 'restaurant_id', 'is_delivering_now']].groupby(['city', 'is_delivering_now']).nunique().sort_values('restaurant_id', ascending=False).reset_index()
df_aux = df_aux[df_aux['is_delivering_now'] == 1]
fig = px.bar(df_aux, x='city', y='restaurant_id',text_auto=True, title='Restaurants that do deliveries per city', labels={'city': 'Cities', 'restaurant_id':'Restaurants'}) 
fig.show()

### Wich city has the largest number of restaurants that accepts online orders?

In [301]:
df_aux = df1[['city', 'restaurant_id', 'has_online_delivery']].groupby(['city', 'has_online_delivery']).nunique().sort_values('restaurant_id', ascending=False).reset_index()
df_aux = df_aux[df_aux['has_online_delivery'] == 1]
fig = px.bar(df_aux, x='city', y='restaurant_id',text_auto=True, title='Restaurants that accepts online orders per city', labels={'city': 'Cities', 'restaurant_id':'Restaurants'}) 
fig.show()

In [302]:
#-----------------------------------------------------
# Restaurants View
#-----------------------------------------------------

# Restaurant with the largest number of ratings

df_aux = df1[['restaurant_name', 'votes']].groupby('restaurant_name').sum().sort_values('votes', ascending=False).reset_index()
df_aux = df_aux.values.tolist()[0]

df_aux

["Domino's Pizza", 59749]

In [303]:
# Restaurant with the highest mean rating

df_aux = df1[['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=False)
df_aux = df_aux[df_aux['aggregate_rating'] == df_aux['aggregate_rating'].max()].sort_values(by='restaurant_id')
df_aux = df_aux.values.tolist()[0][1:3]
df_aux

['Indian Grill Room', 4.9]

In [304]:
# Restaurant that has the most expansive average cost for two

df_aux = df1[['restaurant_name', 'average_cost_for_two']].groupby('restaurant_name').mean().sort_values('average_cost_for_two', ascending=False).reset_index()
df_aux = df_aux.values.tolist()[0]
df_aux

["d'Arry's Verandah Restaurant", 25000017.0]

In [305]:
# Brazilian cuisine restaurant that has the lowest rating.

df_aux = df1[df1['rating_text'] != 'Not rated']
df_aux = df_aux[(df_aux['cuisines'] == 'Brazilian')][['restaurant_name','aggregate_rating']].sort_values('aggregate_rating').reset_index()
df_aux = df_aux.values.tolist()[0][1:]
df_aux


['Café das Estrelas', 3.0]

In [306]:
# Brazilian cuisine brazilian restaurant that has the highest rating.

df_aux = df1[(df1['cuisines'] == 'Brazilian') & (df1['country_name'] == 'Brazil')][['restaurant_name', 'aggregate_rating']].sort_values('aggregate_rating', ascending=False).reset_index()
df_aux = df_aux.values.tolist()[0][1:]
df_aux

['Braseiro da Gávea', 4.9]

In [307]:
# Are the restaurants that accept online orders also, on average, the restaurants with the most registered reviews?

df_aux = df1[df1['has_online_delivery'] == 1][['votes']].mean()
df_aux1 = df1[df1['has_online_delivery'] == 0][['votes']].mean()
df_aux > df_aux1

votes    True
dtype: bool

In [308]:
# The restaurants that take reservations are also, on average, the restaurants that have the highest average value for a meal for two people?

df_aux = df1[df1['has_table_booking'] == 1][['average_cost_for_two']].mean()
df_aux1 = df1[df1['has_table_booking'] == 0][['average_cost_for_two']].mean()

df_aux > df_aux1

average_cost_for_two    True
dtype: bool

In [309]:
# Do Japanese cuisine restaurants in the United States of America have a higher average value for a meal for two people compared to American barbecue (BBQ) restaurants?

df_aux = df1[(df1['cuisines'] == 'Japanese') & (df1['country_name'] == 'United States of America')][['average_cost_for_two']].mean()
df_aux1 = df1[(df1['cuisines'] == 'BBQ') & (df1['country_name'] == 'United States of America')][['average_cost_for_two']].mean()

df_aux > df_aux1

average_cost_for_two    True
dtype: bool

In [310]:
#-----------------------------------------------------
# Cuisines View
#-----------------------------------------------------

# Among the restaurants with Italian cuisine, what is the name of the restaurant with the highest average rating?

df_aux = df1[df1['cuisines'] == 'Italian'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(by='aggregate_rating', ascending=False)
df_aux = df_aux[df_aux['aggregate_rating'] == df_aux['aggregate_rating'].max()].sort_values(by='restaurant_id')
df_aux = df_aux.values.tolist()[0][1:3]
df_aux

['Darshan', 4.9]

In [311]:
# Among the restaurants with Italian cuisine, what is the name of the restaurant with the lowest average rating?

df_aux = df1[df1['rating_text'] != 'Not rated']
df_aux = df_aux[df1['cuisines'] == 'Italian'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(by='aggregate_rating', ascending=False)
df_aux = df_aux[df_aux['aggregate_rating'] == df_aux['aggregate_rating'].min()].sort_values(by='restaurant_id')
df_aux = df_aux.values.tolist()[0][1:3]
df_aux

["Siena's", 2.3]

In [312]:
# Among the restaurants with American cuisine, what is the name of the restaurant with the highest average rating?

df_aux = df1[df1['cuisines'] == 'American'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(by='aggregate_rating', ascending=False)
df_aux = df_aux[df_aux['aggregate_rating'] == df_aux['aggregate_rating'].max()].sort_values(by='restaurant_id')
df_aux = df_aux.values.tolist()[0][1:3]
df_aux

['Burger & Lobster', 4.9]

In [313]:
# Among the restaurants with American cuisine, what is the name of the restaurant with the lowest average rating?

df_aux = df1[df1['rating_text'] != 'Not rated']
df_aux = df_aux[df1['cuisines'] == 'American'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(by='aggregate_rating', ascending=False)
df_aux = df_aux[df_aux['aggregate_rating'] == df_aux['aggregate_rating'].min()].sort_values(by='restaurant_id')
df_aux = df_aux.values.tolist()[0][1:3]
df_aux

["Guy Fieri's Kitchen & Bar", 2.2]

In [314]:
# Among the restaurants with Arabian cuisine, what is the name of the restaurant with the highest average rating?

df_aux = df1[df1['cuisines'] == 'Arabian'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(by='aggregate_rating', ascending=False)
df_aux = df_aux[df_aux['aggregate_rating'] == df_aux['aggregate_rating'].max()].sort_values(by='restaurant_id')
df_aux = df_aux.values.tolist()[0][1:3]
df_aux

['Mandi@36', 4.7]

In [315]:
# Among the restaurants with Arabian cuisine, what is the name of the restaurant with the lowest average rating?

df_aux = df1[df1['rating_text'] != 'Not rated']
df_aux = df_aux[df1['cuisines'] == 'Arabian'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(by='aggregate_rating', ascending=False)
df_aux = df_aux[df_aux['aggregate_rating'] == df_aux['aggregate_rating'].min()].sort_values(by='restaurant_id')
df_aux = df_aux.values.tolist()[0][1:3]
df_aux

['Empório Árabe', 2.7]

In [316]:
# Among the restaurants with Japanese cuisine, what is the name of the restaurant with the highest average rating?

df_aux = df1[df1['cuisines'] == 'Japanese'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(by='aggregate_rating', ascending=False)
df_aux = df_aux[df_aux['aggregate_rating'] == df_aux['aggregate_rating'].max()].sort_values(by='restaurant_id')
df_aux = df_aux.values.tolist()[0][1:3]
df_aux

['Sushi Samba', 4.9]

In [317]:
# Among the restaurants with Japanese cuisine, what is the name of the restaurant with the lowest average rating?

df_aux = df1[df1['rating_text'] != 'Not rated']
df_aux = df_aux[df1['cuisines'] == 'Japanese'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(by='aggregate_rating', ascending=False)
df_aux = df_aux[df_aux['aggregate_rating'] == df_aux['aggregate_rating'].min()].sort_values(by='restaurant_id')
df_aux = df_aux.values.tolist()[0][1:3]
df_aux

['Orienthai', 2.4]

In [318]:
# Among the restaurants with Home-made cuisine, what is the name of the restaurant with the highest average rating?

df_aux = df1[df1['cuisines'] == 'Home-made'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(by='aggregate_rating', ascending=False)
df_aux = df_aux[df_aux['aggregate_rating'] == df_aux['aggregate_rating'].max()].sort_values(by='restaurant_id')
df_aux = df_aux.values.tolist()[0][1:3]
df_aux

['Kanaat Lokantası', 4.0]

In [319]:
# Among the restaurants with Home-made cuisine, what is the name of the restaurant with the lowest average rating?

df_aux = df1[df1['rating_text'] != 'Not rated']
df_aux = df_aux[df1['cuisines'] == 'Home-made'][['restaurant_id', 'restaurant_name', 'aggregate_rating']].sort_values(by='aggregate_rating', ascending=False)
df_aux = df_aux[df_aux['aggregate_rating'] == df_aux['aggregate_rating'].min()].sort_values(by='restaurant_id')
df_aux = df_aux.values.tolist()[0][1:3]
df_aux

['GurMekan Restaurant', 3.7]

In [320]:
# "What type of cuisine has the highest average value for a meal for two people?"

df_aux = df1[['cuisines', 'average_cost_for_two']].sort_values(by='average_cost_for_two', ascending=False).values.tolist()[0][0]
df_aux

'Modern Australian'

In [321]:
# "What type of cuisine has the highest average rating?

df_aux = df1[['cuisines', 'aggregate_rating']].sort_values(by='aggregate_rating', ascending=False).values.tolist()[0][0]
df_aux

'Italian'

In [322]:
# What type of cuisine has the most restaurants that accept online orders and offer delivery services?

df_aux = df1[df1['has_online_delivery'] & df1['is_delivering_now'] == 1][['restaurant_id', 'cuisines']]
df_aux = df_aux.groupby('cuisines').nunique().sort_values(by='restaurant_id', ascending=False).reset_index().values.tolist()[0][0]

In [323]:
df_aux

'North Indian'